In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

dataset = helpers.load_adult_income_dataset()

In [2]:
#INPUT: data, model, cont_feat, output
feature_names=['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'gender', 'hours_per_week']
cont_feat=['age','hours_per_week']
outcome_name='income'

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Checkbox, VBox,HBox,Label, interact,Box, FloatSlider, interact_manual

displayselectchildren=[Label('1. Select Features to Use')]
feature_children=[]
for i in feature_names:
    new_check1=Checkbox(description=i, value=True, indent=False)
    feature_children.append(new_check1)
    displayselectchildren.append(new_check1)

feature_vb = VBox(children = feature_children)
displayselect=VBox(children=displayselectchildren)


def f(**kwargs):
    use = list(kwargs.values())
    
    using=[]
    for i in feature_names:
        using.append(i)
        
    for i in range(len(use)):
        if use[i] is False:
            using.remove(feature_names[i])
            
    #LABELS
    labels=[]
    for i in using:
        labels.append(Label(i))
    justlabels=VBox(children=labels)
            
    #FEATURE WEIGHTS
    
    
    b=widgets.Dropdown(options=['','Use Default Weights','Choose Your Own Weights'],value='',
                       description='2. Feature Weights:',style = {'description_width': 'initial'})
    
    def g(i):
        if i=='Choose Your Own Weights':
            feat_child=[]
            sliders=[]
            for i in using:
                new_slider=widgets.FloatText(value=1,description=i)
                sliders.append(new_slider)

            #justslider=VBox(children=sliders)
            #slidersandlabels=HBox(children=[justlabels,justslider])
            #feat_child.append(slidersandlabels)

            #weights=VBox(children=feat_child)
            weights=VBox(children=sliders)
            display(weights)
        elif i=='Use Default Weights':
            print('Using Default Weights!!')
        elif i=='':
            print('')
            
    out1 = widgets.interactive_output(g, {'i': b})
    feat=VBox(children=[b,out1])
    
    #INPUT QUERY
    
    a=widgets.Dropdown(
    options=['', 'From Dataset', 'Manually Enter'],
    value='',
    description='3. Query Input:',style = {'description_width': 'initial'})

    def h(b):
        if b=='From Dataset':
            automatic_entry=widgets.IntText(value=1,description='Enter row number of query:',
                                         style = {'description_width': 'initial'})

            display(automatic_entry)

        elif b=='Manually Enter':
            inputq=[]

            inputs=[]
            index=-1
            for i in using:
                index+=1
                if i in cont_feat:
                    cont_text=widgets.FloatText(value=0)
                    inputs.append(cont_text)

                else:
                    noncont_dropdown=widgets.Dropdown(options=dataset[i].unique())#description=i,
                    inputs.append(noncont_dropdown)

            justinputs=VBox(children=inputs)
            inputsandlabels=HBox(children=[justlabels,justinputs])

            inputq.append(inputsandlabels)
            query=VBox(children=inputq)
            display(query)

        elif b=='':
            print('')

    out2 = widgets.interactive_output(h, {'b': a})
    query1=VBox(children=[a,out2])
    
    #DISPLAYING FIRST 3
        
    display(HBox([displayselect,feat,query1]))
    
            
args = {
    f'arg{i}': child
    for i, child in enumerate(feature_children) 
}
out=widgets.interactive_output(f, args)
display(out)






# PROX/DIV WEIGHTS
prox = widgets.FloatSlider(description='Proximity Weight', max=10, value=0.5)#default for prox in DiCE is 0.5
div = widgets.FloatSlider(description='Diversity Weight', max=10, value=1)#default for div in DiCE is 1

vb_proxdiv = VBox(children = [prox, div])
tune_proxdiv = Checkbox(description='4. Tune proximity/diversity?', indent=False)

def add_2(button):
    if button['new']:
        vb_proxdiv.children = [prox, div]
        display(vb_proxdiv)
    else:
        vb_proxdiv.children=[]

tune_proxdiv.observe(add_2, names='value')

#NUMBER OF EXPLANATIONS
num_exp = widgets.IntSlider(description='5. Num of Explanations', min=1, max=5,style = {'description_width': 'initial'}, indent=False)
  
#HIGHLIGHT CHANGEs
highlight_changes=widgets.Checkbox(
    description='6. Only highlight changes in explanations?',
    disabled=False,
    indent=False)

       
#DISPLAYING LAST 3 WIDGETS
display(HBox([tune_proxdiv, num_exp, highlight_changes]))


# Generating Counterfactuals

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Checkbox, VBox,HBox,Label, interact,Box, FloatSlider, interact_manual

displayselectchildren=[Label('1. Select Features to Use')]
feature_children=[]
for i in feature_names:
    new_check1=Checkbox(description=i, value=True, indent=False)
    feature_children.append(new_check1)
    displayselectchildren.append(new_check1)

feature_vb = VBox(children = feature_children)
displayselect=VBox(children=displayselectchildren)

def f(**kwargs):
    use = list(kwargs.values())
    
    using=[]
    for i in feature_names:
        using.append(i)
        
    for i in range(len(use)):
        if use[i] is False:
            using.remove(feature_names[i])
            
    f.useusing=using
    
    #LABELS
    labels=[]
    for i in using:
        labels.append(Label(i))
    justlabels=VBox(children=labels)
            
    #FEATURE WEIGHTS
    
    b=widgets.Dropdown(options=['','Use Default Weights','Choose Your Own Weights'],value='',
                       description='2. Feature Weights:',style = {'description_width': 'initial'})
    
    
    def g(i):
        if i=='Choose Your Own Weights':
            feat_child=[]
            floattexts=[]
            for i in using:
                new_floattext=widgets.FloatText(value=1,description=i)
                floattexts.append(new_floattext)

            #justslider=VBox(children=sliders)
            #slidersandlabels=HBox(children=[justlabels,justslider])
            #feat_child.append(slidersandlabels)

            #weights=VBox(children=feat_child)
            weights=VBox(children=floattexts)
            display(weights)
            #putting weights into dict
            floattexts_values=[]
            for i in floattexts:
                floattexts_values.append(i.value)
            weightstouse=dict(zip(using, floattexts_values))
            f.useweights=weightstouse
            
        elif i=='Use Default Weights':
            print('Using Default Weights!')
        elif i=='':
            print('')
            
    out1 = widgets.interactive_output(g, {'i': b})
    feat=VBox(children=[b,out1])
    
    #INPUT QUERY
    
    a=widgets.Dropdown(
    options=['', 'From Dataset', 'Manually Enter'],
    value='',
    description='3. Query Input:',style = {'description_width': 'initial'})

    def h(b):
        if b=='From Dataset':
            automatic_entry=widgets.IntText(value=1,description='Enter row number of query:',
                                         style = {'description_width': 'initial'})

            display(automatic_entry)
            
            #making query into a dict for automatic
            rowdata=[]
            for i in dataset.loc[ automatic_entry.value , : ]:
                rowdata.append(i)

            query_instance = dict(zip(using, rowdata))
            f.usequery=query_instance

        elif b=='Manually Enter':
            inputq=[]

            inputs=[]
            index=-1
            for i in using:
                index+=1
                if i in cont_feat:
                    cont_text=widgets.FloatText(value=0)
                    inputs.append(cont_text)

                else:
                    noncont_dropdown=widgets.Dropdown(options=dataset[i].unique())
                    inputs.append(noncont_dropdown)

            justinputs=VBox(children=inputs)
            inputsandlabels=HBox(children=[justlabels,justinputs])

            inputq.append(inputsandlabels)
            query=VBox(children=inputq)
            display(query)
            
            
            #making query into a dict for manual
            query_values=[]
            for i in inputs:
                query_values.append(i.value)

            query_instance = dict(zip(using, query_values))
            f.usequery=query_instance
        elif b=='':
            print('')
            

    out2 = widgets.interactive_output(h, {'b': a})
    query1=VBox(children=[a,out2])
    
    #DISPLAYING FIRST 3
        
    display(HBox([displayselect,feat,query1]))
    
    
            
args = {
    f'arg{i}': child
    for i, child in enumerate(feature_children) 
}
out=widgets.interactive_output(f, args)
display(out)




# PROX/DIV WEIGHTS
prox = widgets.FloatSlider(description='Proximity Weight', max=10, value=0.5)#default for prox in DiCE is 0.5
div = widgets.FloatSlider(description='Diversity Weight', max=10, value=1)#default for div in DiCE is 1

vb_proxdiv = VBox(children = [prox, div])
tune_proxdiv = Checkbox(description='4. Tune proximity/diversity?', indent=False)

def add_2(button):
    if button['new']:
        vb_proxdiv.children = [prox, div]
        display(vb_proxdiv)
    else:
        vb_proxdiv.children=[]

tune_proxdiv.observe(add_2, names='value')

#NUMBER OF EXPLANATIONS
num_exp = widgets.IntSlider(description='5. Num of Explanations', min=1, max=5,
                            style = {'description_width': 'initial'}, indent=False)
  
#HIGHLIGHT CHANGEs
highlight_changes=widgets.Checkbox(
    description='6. Only highlight changes in explanations?',
    value=True,
    disabled=False,
    indent=False)

       
#DISPLAYING LAST 3 WIDGETS
display(HBox([tune_proxdiv, num_exp, highlight_changes]))


Output()

In [7]:
print(f.useusing)
print(f.usequery)
print(f.useweights)

['age', 'education', 'marital_status', 'occupation', 'race', 'gender']
{'age': 0.0, 'education': 'Bachelors', 'marital_status': 'Single', 'occupation': 'White-Collar', 'race': 'White', 'gender': 'Male'}
{'age': 1.0, 'education': 1.0, 'marital_status': 1.0, 'occupation': 1.0, 'race': 1.0, 'gender': 1.0}


In [ ]:
#SETTING UP
d = dice_ml.Data(dataframe=dataset, continuous_features=cont_feat, outcome_name=outcome_name)

backend = 'TF'+tf.__version__[0] # TF2
m = dice_ml.Model(model=modelname, backend=backend)

exp = dice_ml.Dice(d, m)

#Generating CFs

if b.value=='Use Default Weights':
    dice_exp = exp.generate_counterfactuals(query_instance,total_CFs=num_exp.value, desired_class="opposite",
                                            features_to_vary=f.useusing, 
                                            proximity_weight=prox.value, diversity_weight=div.value)
elif b.value=='Choose Your Own Weights':
    dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=num_exp.value, desired_class="opposite",
                                            features_to_vary=f.useusing, feature_weights=total[0][0],
                                            proximity_weight=prox.value, diversity_weight=div.value)
                                            

#Visualizing
if highlight_changes is True:
    dice_exp.visualize_as_dataframe(show_only_changes=True)
else:
    dice_exp.visualize_as_dataframe()
      

In [ ]:
def a(b):
    b+=2
    a.test = b
    def c(d):
        if d>10:
            d+=1
            a.test2=d
        elif d<10:
            d-=1
            a.test2=d
        else:
            print('yay')
    c(9)
a(4)

print(a.test2)